In [1]:
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
import sys
import cv2 as cv
import numpy as np
import time
import os

class Resolution(QMainWindow):
    def __init__(self):
        super().__init__()
        
        self.setWindowTitle("Make High-Resolution IMG")
        self.setGeometry(200,200,420,200) # 화면에 나타날 윈도우의 위치와 크기를 지정
        
        self.inittext = QLabel("저해상도 이미지를 고해상도로 바꾸어 주는 프로그램", self)
        self.inittext.move(10, 10)
        self.inittext.resize(500, 20)
        
        imgopenButton = QPushButton('이미지 열기', self) # 버튼 생성
        videoopenButton = QPushButton('비디오 열기', self)
        convertButton = QPushButton('영상 변환', self)
        exportButton = QPushButton('내보내기', self)
        quitButton = QPushButton('나가기', self)
        
        self.scaletext = QLabel("Scale (2,4,8 만 가능) : ", self)
        self.scaletext.move(10, 90)
        self.scaletext.resize(300, 20)
        
        self.scale = QLineEdit("", self)
        self.scale.move(165, 90)
        self.scale.resize(30,20)
        self.scale.textChanged.connect(self.lineEdit_textChanged) # self.scale.text()
 
        self.radio1 = QRadioButton("Bicubic(고전방법)", self)
        self.radio1.move(10, 110)
        self.radio1.resize(500,20)
        self.radio1.setChecked(True)
        self.radio1.clicked.connect(self.radioButton_clicked) 
        
        self.radio2 = QRadioButton("Super-Resolution(딥러닝)", self)
        self.radio2.move(10, 130)
        self.radio2.resize(500,20)
        self.radio2.clicked.connect(self.radioButton_clicked)
        
        
 
        imgopenButton.setGeometry(10, 30, 100, 30)
        videoopenButton.setGeometry(10, 60, 100, 30)
        convertButton.setGeometry(110, 30, 100, 30)
        exportButton.setGeometry(210, 30, 100, 30)
        quitButton.setGeometry(310, 30, 100, 30)
        
        imgopenButton.clicked.connect(self.IMGOpenFunction)
        
        convertButton.clicked.connect(self.convertFucntion)
        
        quitButton.clicked.connect(self.quitFunction)
    
    def IMGOpenFunction(self):
        fname = QFileDialog.getOpenFileName(self, 'Open file', './examples') 
        fname = './examples/' + fname[0].split('/')[-1] # 한글 경로 오류로 수동 지정
        print(fname)
        self.img = cv.imread(fname)
        if self.img is None : sys.exit('파일을 찾을 수 없습니다.')
        
        # self.img_show = np.copy(self.img)
        cv.imshow('Original', self.img)
    
    def lineEdit_textChanged(self):
        pass
        #self.statusBar.showMessage(self.lineEdit.text())

    def convertFucntion(self):
        if self.scale.text() == '2':
            model = "models/LapSRN_x2.pb"
        elif self.scale.text() == '4':
            model = "models/LapSRN_x4.pb"
        elif self.scale.text() == '8':
            model = "models/LapSRN_x8.pb"
        
        modelName = "lapsrn" # model.split(os.path.sep)[-1].split("_")[0].lower()
        modelScale = int(self.scale.text())
         
        sr = cv.dnn_superres.DnnSuperResImpl_create()
        sr.readModel(model)
        sr.setModel(modelName, modelScale)
        
        
        if self.method == "bicubic":
            print("bicubic")
            start = time.time()
            self.bicubic = cv.resize(self.img, int(self.img.shape[1])*modelScale, int(self.img.shape[0])*modelScale)
            end = time.time()
            cv.imshow("Bicubic", self.bicubic)
        
        elif self.method == "super":
            print("super-resolution")
            start = time.time()
            self.upscaled = sr.upsample(self.img)
            end = time.time()
            cv.imshow("Super Resolution", self.upscaled)
       
#    def scale_set(self):
#        self.scale
       
    def radioButton_clicked(self):
        self.method = ""
        if self.radio1.isChecked():
            self.method = "bicubic"
        else:
            self.method = "super"
            
    def quitFunction(self):
        cv.destroyAllWindows()
        self.close()
        
app = QApplication(sys.argv)
win = Resolution()
win.show()
app.exec_()
        

./examples/zebra.png


AttributeError: 'Resolution' object has no attribute 'method'

bicubic


error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'resize'
> Overload resolution failed:
>  - Can't parse 'dsize'. Input argument doesn't provide sequence protocol
>  - Can't parse 'dsize'. Input argument doesn't provide sequence protocol


super-resolution
super-resolution


...

AttributeError: 'Resolution' object has no attribute 'model'

AttributeError: 'Resolution' object has no attribute 'model'

AttributeError: 'Resolution' object has no attribute 'model'

AttributeError: 'Resolution' object has no attribute 'model'

0

In [2]:
pwd

'g:\\내 드라이브\\Github\\Make_high-resolution_IMG'